In [ ]:


# Passo 1: Importar as bibliotecas necessárias
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import io, color, filters, exposure
from skimage.util import img_as_ubyte, img_as_float

# --- SEÇÃO DE CONFIGURAÇÃO ---
# Lista de imagens que vamos processar.
# Cada item é uma tupla com (pasta_de_entrada, nome_do_arquivo)
IMAGENS_PARA_PROCESSAR = [
    ('Imagens_atividades', 'FIG033_4.TIF'),
    ('Imagens', 'chave_i_n_01.jpg') # Exemplo da sua base de imagens
]

# Pasta onde salvaremos os resultados
PASTA_SAIDA = 'Imagens_Laplaciano'
# --------------------------------

# Passo 2: Preparar a pasta de saída
print(f"Verificando/Criando a pasta de destino: {PASTA_SAIDA}")
os.makedirs(PASTA_SAIDA, exist_ok=True)

# Passo 3: Loop para processar cada imagem da lista
for pasta_entrada, nome_imagem in IMAGENS_PARA_PROCESSAR:
    print(f"\n--- Processando a imagem: {nome_imagem} ---")
    
    # Monta o caminho completo da imagem de entrada
    caminho_entrada = os.path.join(pasta_entrada, nome_imagem)
    
    # Carrega a imagem
    imagem_original = io.imread(caminho_entrada)
    
    # Verifica se a imagem já é cinza antes de tentar converter
    if imagem_original.ndim == 3:
        imagem_cinza_float = img_as_float(color.rgb2gray(imagem_original))
    else:
        imagem_cinza_float = img_as_float(imagem_original)
    
    # --- APLICAÇÃO DA TÉCNICA ---
    
    # 1. Aplicar o filtro Laplaciano
    resultado_laplaciano = filters.laplace(imagem_cinza_float)
    
    # 2. Aguçar a imagem (Sharpening)
    imagem_agucada = imagem_cinza_float - resultado_laplaciano
    
    # CORREÇÃO: Garante que os valores dos pixels fiquem na faixa válida [0, 1]
    imagem_agucada = np.clip(imagem_agucada, 0, 1)

    # --- SALVANDO AS IMAGENS RESULTANTES ---
    print("Salvando imagens...")
    nome_base = nome_imagem.split('.')[0]
    
    laplaciano_salvavel = img_as_ubyte(exposure.rescale_intensity(resultado_laplaciano, out_range=(0, 1)))
    io.imsave(os.path.join(PASTA_SAIDA, f'{nome_base}_laplaciano.jpg'), laplaciano_salvavel)

    io.imsave(os.path.join(PASTA_SAIDA, f'{nome_base}_agucada.jpg'), img_as_ubyte(imagem_agucada))

    # --- VISUALIZAÇÃO DOS RESULTADOS ---
    print("Exibindo resultados...")
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    axes[0].imshow(imagem_cinza_float, cmap='gray')
    axes[0].set_title(f'Original: {nome_imagem}')
    axes[0].axis('off')

    axes[1].imshow(resultado_laplaciano, cmap='gray')
    axes[1].set_title('Resultado do Filtro Laplaciano')
    axes[1].axis('off')

    axes[2].imshow(imagem_agucada, cmap='gray')
    axes[2].set_title('Imagem Aguçada (Sharpened)')
    axes[2].axis('off')

    plt.show()

print("\n\nProcesso finalizado!")